In [219]:
#Step 0: Restarting the session and Clearning all temporary variables----------------------------

try:
    from IPython import get_ipython
    get_ipython().magic('clear')
    get_ipython().magic('reset -f')
except:
    pass

In [220]:
# step 1 : import libraries 

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame
import pandas_datareader as pdr
from matplotlib import pyplot as plt
import math
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.holiday import Holiday, USMemorialDay, AbstractHolidayCalendar, nearest_workday, MO
import seaborn as sns
from scipy.stats import norm, linregress
import statsmodels.formula.api as smf
#import empiricaldist
from sqlalchemy import create_engine
from zlib import crc32

In [221]:
# step 2 : create a function that from each Whatsapp message file, gets the phone number of the users being active

def cellphone(file):
    """
    :param file: name of the whatspp file after been read by pd.read_csv()
    :return: a Series of phonenumbers of the users in a whatsapp chat group.
    """

    # take the number of rows assing it to n
    n = file.shape[1]
    
    #create a name for dataframe columns (here col names are numbers)
    file.columns=list(range(n))
    
    # retrieve the first column (there is almost nothing in other columns)
    file_col=file[0]

    # iterate in each row to split the words (there is no space between phone numbers). They are considered as a word!
    s=[]
    for i in range(file.shape[0]):
        s.append(str(file_col.iloc[i]).split(' '))
    
    #iterate over each item in 's' and take any word that begins with '+' and ends in ':'
    m=[]
    for j in s:
        for i in j:
            if ':' and '+98' in i:
                m.append(i.replace(':', ''))

    #convert the list of phone numbers into a DF, drop duplicates and reset index.
    tel = pd.DataFrame(m).drop_duplicates().reset_index()
    return tel


In [222]:
# step 3 : retrieve the phonenumber of ACTIVE users from Whatsapp groups (mentioned in below): 

#name of whatsApp files
string = ['-chat 0.txt', '-chat 1.txt','-chat 2.txt', '-chat 3.txt', '-chat 4.txt', '-chat 5.txt', '-chat 6.txt', '-chat 7.txt']

#using 'cellphone()' function defined earlier, we get the numbers of each file
d={}

for i in range(len(string)):
    file = cellphone(pd.read_csv(string[i]))
    d[i]=file[0]

#convert the dictionary containg numbers for each file into a DataFrame (each col in the df will correspond to a file)
df= pd.DataFrame(d)

#contactenate all the cols of the DataFrame
dg=pd.concat([df[0], df[1],df[2],df[3],df[4],df[5],df[6], df[7]], axis=0)

#reset the index and drop NaNs
dg= dg.reset_index().dropna()

#give the name 'Phone' to the col containg phone numbers
dg.rename(columns={0:'Phone'}, inplace=True)

#drop the col called 'index' after reindexing
dg= dg.drop(['index'], axis=1)
print(dg)

#save the file into a .csv
dg.to_csv('phone_num.csv', index=False)

                   Phone
0     ‪+98 912 833 8508‬
1     ‪+98 919 216 4319‬
2     ‪+98 912 811 5488‬
3     ‪+98 937 100 1288‬
4     ‪+98 922 502 4320‬
..                   ...
654  ‎‪+98 991 565 7994‬
655  ‎‪+98 930 673 6072‬
656  ‎‪+98 912 376 1369‬
657  ‎‪+98 939 586 1061‬
658  ‎‪+98 917 623 9945‬

[369 rows x 1 columns]


In [223]:
# step 4 : retrieve the phonenumber of ALL users from Whatsapp groups (mentioned in below):

string = ['group_01.csv', 'group_02.csv','group_03.csv', 'group_04.csv', 'group_05.csv', 'group_06.csv']

d={}

for i in range(len(string)):
    file = pd.read_csv(string[i])
    d[i]=file['Phone']

df= pd.DataFrame(d)
dg=pd.concat([df[0], df[1],df[2],df[3],df[4],df[5]], axis=0)


dg= dg.reset_index().dropna()
dg.rename(columns={0:'Phone'}, inplace=True)

dg= dg.drop(['index'], axis=1)

print(dg)
dg.to_csv('phone_num_all.csv', index=False)

            Phone
0    3.365682e+10
1    9.891214e+11
2    9.891091e+11
3    9.891001e+11
4    9.890173e+11
..            ...
653  9.891666e+11
654  9.893589e+11
655  9.893719e+11
656  9.891773e+11
657  9.892250e+11

[515 rows x 1 columns]


In [260]:
phone_active = pd.read_csv('phone_num.csv')
phone_all=pd.read_csv('phone_num_all.csv')

phone_active.drop_duplicates(keep='first', inplace=True)
phone_all.drop_duplicates(keep='first', inplace=True)

print(phone_active.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 365
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Phone   241 non-null    object
dtypes: object(1)
memory usage: 3.8+ KB
None


In [261]:
# strip everything that is not digit

phone_active['Phone']=phone_active['Phone'].str.replace(r'\D+','')

phone_all=phone_all.astype(str)
phone_all['Phone']=phone_all['Phone'].str.replace(r'\D+','')

#get only 12 charachters from phone_all numbers
phone_all['Phone']=phone_all['Phone'].str[:12]


phone_all= phone_all.reset_index()
phone_active = phone_active.reset_index()



In [262]:
# identify non-conform numbers : those that don't begin with '98' or those which are less than 12 digits

aa=[]
for i in range(phone_active.shape[0]):
    if len(phone_active['Phone'].iloc[i])<12:
        aa.append(i)
        print(i, phone_active['Phone'].iloc[i])
        
for i in range(phone_active.shape[0]):
    if phone_active['Phone'].iloc[i][:2] !='98':
        aa.append(i)
        print(i, phone_active['Phone'].iloc[i])
        
print(aa)


45 98
102 98
105 98
[45, 102, 105]


In [263]:
bb=[]

for i in range(phone_all.shape[0]):
    if len(phone_all['Phone'].iloc[i])<12:
        bb.append(i)
        print(i, phone_all['Phone'].iloc[i])
        
for i in range(phone_all.shape[0]):
    if phone_all['Phone'].iloc[i][:2] !='98':
        bb.append(i)
        print(i, phone_all['Phone'].iloc[i])
        
        
print(bb)

0 336568176890
5 447983246758
6 447570804898
7 316218745220
9 447871731376
57 447909725682
58 393318971569
228 447711812788
[0, 5, 6, 7, 9, 57, 58, 228]


In [264]:
# drop non-conform numbers
phone_active.drop(aa, inplace=True)
phone_all.drop(bb, inplace=True)

In [265]:
print('Number of active users in whatsapp groups since the creation of groups:', phone_active.shape[0])


Number of active users in whatsapp groups: 238


In [266]:
print('Number of all users in whatsapp groups:', phone_all.shape[0])

Number of all users in whatsapp groups: 356


In [267]:
#calculate the precentage
percentage= phone_active.shape[0]*100 / phone_all.shape[0]

print(f'Percentage of whatsapp active users:{round(percentage,2)} %')

Percentage of whatsapp active users:66.85 %


In [268]:
df_merge= phone_all.merge(phone_active, on='Phone', how='inner', indicator=False)
df_merge= df_merge.drop_duplicates()
print('Number of active users that are still present in groups:', df_merge.shape[0])

Number of active users that are still present in groups: 151


In [269]:
print('Number of users that have left groups (min): ', phone_active.shape[0]-df_merge.shape[0])

Number of users that have left groups (min):  87


In [270]:
# Reading the phone numbers from website dataset
df= pd.read_excel('orders_3.xlsx')
df['Phone']=980000000000 +df['Phone (Billing)']
df['phone']= df['Phone'].astype(str)

dg =df['Phone'].dropna()
dg= dg.reset_index()
dg = dg.drop_duplicates()

print(dg)

      index         Phone
0         0  989133260467
1         1  989188311169
2         2  989177106525
3         3  989126956235
4         4  989133173190
...     ...           ...
1427   1427  989027675649
1428   1428  989123140587
1429   1429  989307876365
1430   1430  989120495422
1431   1431  989120495422

[1432 rows x 2 columns]


In [271]:
dg=dg.astype(str)
print(dg.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1432 entries, 0 to 1431
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   1432 non-null   object
 1   Phone   1432 non-null   object
dtypes: object(2)
memory usage: 33.6+ KB
None


In [272]:
df_dg=dg.merge(phone_all, on='Phone', how='inner')
df_dg=df_dg['Phone'].drop_duplicates()

In [273]:
df_dg= df_dg.reset_index()

In [274]:
print('Number of whatsapp users (phone numbers) who have done shopping from website:', df_dg.shape[0])

Number of whatsapp users (phone numbers) who have done shopping from website: 117


In [276]:
x = df_dg.shape[0] *100/ phone_all.shape[0]
print('percentage of clients coming from Whatsapp:', round(x,2))

percentage of clients coming from Whatsapp: 32.87


In [277]:
y = df_dg.shape[0] *100/ phone_active.shape[0]
print('percentage of clients coming from Whatsapp active clients:', round(y,2))

percentage of clients coming from Whatsapp active clients: 49.16
